In [49]:
pwd

'/Users/ozkansafak/code/fraktur/notebooks'

In [58]:
%%time
%load_ext autoreload
%autoreload 2

# import standard Python modules
import time 
import numpy as np
import os, sys
import re
import glob
import openai

# # Get the root path of the project
sys.path.append(os.path.abspath(".."))

# Import necessary modules from your src directory
from src.utils import timeit, encode_image
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload, send_gpt_request
from src.document_generation import create_docx
from src.api_requests import single_page

from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% !important; }</style>"))

pylab.rcParams.update(
    {
        "legend.fontsize": "small",
        "font.size": 12,
        "figure.figsize": (6*3, 2.2*3),
        "axes.labelsize": "small",
        "axes.titlesize": "medium",
        "axes.grid": "on",
        "xtick.labelsize": "small",
        "ytick.labelsize": "small",
    }
)

print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<timed exec>:22: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 

CPU times: user 1.72 ms, sys: 1.2 ms, total: 2.92 ms
Wall time: 2.31 ms


---
## Fraktur Translator

In [59]:
# Set the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Set the headers for the API request
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

model_name = "gpt-4o-mini-2024-07-18" # ["gpt-4o-2024-08-06", "gpt-4o-mini-2024-07-18"]
plotter = False


In [121]:
# Folder containing the Fraktur PDF files
image_path = "../Der Weltkrieg v8 Chap 4F (in Fraktur)/*pdf"

# Get the list of PDF file paths
fnames = sorted(glob.glob(image_path))
german_texts = {}
english_texts = {}

for i, fname in enumerate(fnames[3:4]):
    total_time = 0    
    start = time.time()

    pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
    print(f"{'-'*50}\npage:{pageno} ({i}/{len(fnames)-1}) ",end="")
    
    raw_german_text, german_text, english_text = single_page(fname, model_name, headers, plotter, pageno)
    
    german_texts[pageno] = german_text
    english_texts[pageno] = english_text
    delta = time.time() - start 
    total_time += delta
    print(f"time:{delta:.2f} sec\n{'-'*50}")
    print(f"<raw_german_text>{raw_german_text}</raw_german_text>\n")
    print(f"<german_text>{german_text}</german_text>\n")
    print(f"<english_text>{english_text}</english_text>")
    print(f"\n[Runtime {int(total_time//60)} mins {(total_time - int(total_time//60)*60):.2f} sec]") 


--------------------------------------------------
page:459 (0/83) time:27.61 sec
--------------------------------------------------
<raw_german_text>und 41. Infanterie-Division fuhren bei Alt Aus auf starken Feind, gegen ben die Entscheidung auf den folgenden Tag verschoben wurde. Inzwischen konnte das Kavalleriekorps Schmettoff etwa zwölf Kilometer nordwestlich des Ortes nachmittags starke russische Kavallerie zurückdrängen und fiel in die Vorderlaces des Gegners vorbringens; die vordersten Teile der 6. Kavallerie-Division kamen dabei bis Dubben, standen also bereits mehr als 30 Kilometer östnördöstlich von Alt Aus. Mehrere tausend Gefangene und einige Geschütze wurden die Beute dieser ersten Kampfage. Der Versuch, den Gegner bei Alt Aus am 17. Juli durch umfassenden Angriff zu vernichten, glitt nicht, da er unübersehbar starke Kräfte gegen Norden herausgeschoben hatte. Es kam zu ernsten Kämpfe, der ihn notwendig, so eilig nach Osten ausweichen, darf es abermals 4000 Gefangene und ei

In [120]:
text =  {                "text": "This work consists of three distinct subtasks:\n\n"
                        "**Step 1. OCR:** Transcribe the whole page into German and wrap the output in <raw_german></raw_german> tags. "
                        "Pay great attention to transcribing all the text and all the fraktur characters you detect in your OCR work \n\n"
                        "**Step 2. Layout Analysis:**. Look at the image of the page and your transcription again "
                        "and redo the transcription like this: \n"
                        "   - Wrap your second transcription inside <german></german> tags.\n"
                        "   - Stay loyal to the new line characters (\\n) you have detected and transcribed in the OCR step above. "
                        "I want you to divide the parts of the transcription inside the <raw_german></raw_german> tags "
                        "into header, body, and footer sections: \n"
                        "   - If, and only if, there's a header text (like a chapter title or a section heading), "
                        "wrap the it inside <header></header> tags. If there's no header text, "
                        "don't include any <header></header> tags at all. \n"
                        "   - Next, wrap the body of the text inside <body></body> tags. \n"
                        "   - Finally, and if, and only if,  there are is footnotes, wrap them inside <footer></footer> tags. \n\n"
                        "If there's no footer text, don't include any <footer></footer> tags at all. "
                        "**Step 3. Translation DE2EN:**. Translate the German text into English while maintaining the header, "
                        "body, and footer sections.\n\n"
                        "EXAMPLE OUTPUT FORMAT:\n"
                        "<raw_german>.................</raw_german>\n\n"
                        "<german><header>....</header>\n<body>..............</body>\n<footer>......</footer>\n</german>\n\n"
                        "<english>....................</english>\n"
}
print(text['text'])

This work consists of three distinct subtasks:

**Step 1. OCR:** Transcribe the whole page into German and wrap the output in <raw_german></raw_german> tags. Pay great attention to transcribing all the text and all the fraktur characters you detect in your OCR work 

**Step 2. Layout Analysis:**. Look at the image of the page and your transcription again and redo the transcription like this: 
   - Wrap your second transcription inside <german></german> tags.
   - Stay loyal to the new line characters (\n) you have detected and transcribed in the OCR step above. I want you to divide the parts of the transcription inside the <raw_german></raw_german> tags into header, body, and footer sections: 
   - If, and only if, there's a header text (like a chapter title or a section heading), wrap the it inside <header></header> tags. If there's no header text, don't include any <header></header> tags at all. 
   - Next, wrap the body of the text inside <body></body> tags. 
   - Finally, and if, a

In [13]:
folder_name = 'third_attempt'
with open(f'output/{folder_name}/english_texts.json', 'w') as f:
    json.dump(english_texts, f)
with open(f'output/{folder_name}/german_texts.json', 'w') as f:
    json.dump(german_texts, f)

create_docx(english_texts, folder_name, English='English')
create_docx(german_texts, folder_name, English='German')


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

In [23]:
# Define available GPT-4o models with pricing information
"""
models = {
    "gpt-4o-2024-08-06": {
        "price_txt": "$2.50 / 1M input tokens", 
        "price_img": "$0.001913 / 1500px^2"
    },
    "gpt-4o-mini-2024-07-18": {
        "price_txt": "$0.150 / 1M input tokens", 
        "price_img": "$0.003825 / 1500px^2"
    }
}

""";
